<a href="https://colab.research.google.com/github/padmathanumoorthy/gen-ai/blob/main/In_context_learning_Flan_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


Install pyTorch - Torch lib

In [5]:
pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1 --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 903.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torchdata==0.7.0, but you have torchdata 0.5.1 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch

**Install huggigface library**

In [6]:
pip install transformers==4.27.2 datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.9 MB/s eta 0:00:00


**Load the datasets, LLM, tokenizer, configurator.**

In [7]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

# Summarize Dialogue without prompt engineering

Generating the summary of a dialogue with the pre-trained LLM Flan T5 from Huggingface. The list of available models in the HuggingFace *transformers* package can be found here:

Upload simple dialogues from the Dialogsum huggingface dataset.
This contains 10000+dialogue and the corresponding manually labeled summaries and topics.

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"
#public dataset from huggingface
dataset = load_dataset(huggingface_dataset_name)

In [ ]:
example_indices = [80,300,400]
#dash_line = '-'.join('' for x in range(100))
dash_line = '-'*100
for i, index in enumerate(example_indices):
  print(dash_line)
  print('Example ',i+1)
  print(dash_line)
  print('Input Dialogue:')
  print(dataset['test'][index]['dialogue'])
  print(dash_line)
  print('Baseline human summary:')
  print(dataset['test'][index]['summary'])
  print(dash_line)
  print()

In [ ]:
#load the Flan t5 model creating an instance of the AutoModelForSeq2SeqLM class with the from_pretrained method
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
#To perform encoding and decoding, we need to work with text in a tokenized form.
#Tokenization is the process of splitting texts into smaller units that can be
#processed by the LLM models

#Download the tokenizer for the Flan-t5 model using transformer package AutoTokenizer.from_pretrained() method,
#Parameter use_fast switches on fast Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
#Test the AutoTokenizer instance for its encoding and decoding with a simple sentence

dialogue = "Hello Padma Thanumoorthy, How are you doing?"
dialogue_encoded = tokenizer(dialogue, return_tensors='pt')
print('Encoded Dialogue: ')
print(dialogue_encoded)
dialogue_decoded = tokenizer.decode(dialogue_encoded['input_ids'][0],
                                    skip_special_tokens=True)

print(dash_line)
print('Encoded Dialogue Tensor: Weighted numbers of vector embeddings')
print(dialogue_encoded['input_ids'][0])
print(dash_line)
print('Decoded Dialogue: ')
print(dialogue_decoded)
print(dash_line)

In [ ]:
#Now its time to explore how well the base FM LLM summarizes a dialogue withiut any prompt enigneering.
#Prompt Engineering is an act of a human changing the prommpt (input) to improve the response for a given task

for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  inputs = tokenizer(dialogue, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(inputs['input_ids'], max_new_tokens=50)[0],
      skip_special_tokens=True)

  print(dash_line)
  print('Example ',i+1)
  print(dash_line)
  print('Input Dialogue:')
  print(dialogue)
  print(dash_line)
  print('Baseline human summary:')
  print(summary)
  print(dash_line)
  print('Model generation without prompt engineering:')
  print(output)
  print()


# Summarize Dialogue with an Instruction Prompt

PE is an imp concept in using Foundation Models for text generation.

1. Zero Shot inference with an instruction prompt

in order to instruct the model to perform a task - summarize a dialogue - take the dialogue and convert that into an instruction prompt. This is called Zero Shot Inference.





In [ ]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
  Summarize the following conversation.
  {dialogue}

  Summary:
  """

  inputs = tokenizer(prompt, return_tensors='pt')
  output = tokenizer.decode(
      model.generate(inputs['input_ids'], max_new_tokens=50)[0],
      skip_special_tokens=True)

  print(dash_line)
  print('Example ',i+1)
  print(dash_line)
  print('Input Dialogue:')
  print(dialogue)
  print(dash_line)
  print('Baseline human summary:')
  print(summary)
  print(dash_line)
  print('Model generation with Zero Shot inference prompt engineering:')
  print(output)
  print()

In [ ]:
def make_prompt(example_indices_full, example_index_to_summarize):
  prompt = ''
  for index in example_indices_full:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Dialogue:
    {dialogue}

    What was going on?
    {summary}
    """
  dialogue = dataset['test'][example_index_to_summarize]['dialogue']

  prompt += f"""
  Dialogue:
  {dialogue}

  What was going on?
  """
  return prompt

In [ ]:
example_indices_full = [80]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs['input_ids'], max_new_tokens=50)[0],
    skip_special_tokens=True)

print(dash_line)
print('Example ',i+1)
print(dash_line)
print('Input Dialogue:')
print(dialogue)
print(dash_line)
print('Baseline human summary:\n{summary}\n')
print(dash_line)
print('Model generation with a One Shot inference prompt engineering:\n{output}\n')

print()